In [ ]:
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import (
    KMeans, 
    AgglomerativeClustering, 
    DBSCAN, 
    SpectralClustering, 
    Birch, 
    MeanShift, 
    AffinityPropagation
)
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn_extra.cluster import KMedoids
from sklearn.cluster import OPTICS
import hdbscan
import time


In [ ]:

# Generate datasets
def generate_datasets():
    datasets = []
    n_samples = 1500
    random_state = 170
    noisy_circles = make_blobs(n_samples=n_samples, centers=[[0, 0], [1, 1]], random_state=random_state)
    noisy_moons = make_moons(n_samples=n_samples, noise=0.05, random_state=random_state)
    blobs = make_blobs(n_samples=n_samples, random_state=random_state)
    aniso = make_blobs(n_samples=n_samples, random_state=random_state)
    transformation = [[0.6, -0.6], [-0.4, 0.8]]
    aniso = (np.dot(aniso[0], transformation), aniso[1])
    datasets.append(noisy_circles)
    datasets.append(noisy_moons)
    datasets.append(blobs)
    datasets.append(aniso)
    return datasets

# Define clustering algorithms
def define_algorithms():
    algorithms = {
        "MiniBatch KMeans": KMeans(n_clusters=3, random_state=42),
        "Affinity Propagation": AffinityPropagation(random_state=42),
        "MeanShift": MeanShift(),
        "Spectral Clustering": SpectralClustering(n_clusters=3, affinity='nearest_neighbors'),
        "Ward": AgglomerativeClustering(n_clusters=3, linkage='ward'),
        "Agglomerative Clustering": AgglomerativeClustering(n_clusters=3),
        "DBSCAN": DBSCAN(eps=0.2),
        "HDBSCAN": hdbscan.HDBSCAN(min_cluster_size=30),
        "OPTICS": OPTICS(min_samples=10),
        "BIRCH": Birch(n_clusters=3),
        "Gaussian Mixture": GaussianMixture(n_components=3, random_state=42)
    }
    return algorithms

# Plot results
def plot_results(datasets, algorithms):
    fig, axes = plt.subplots(len(datasets), len(algorithms), figsize=(20, 15))
    axes = axes.flatten()
    
    for i_dataset, dataset in enumerate(datasets):
        X, y = dataset
        X = StandardScaler().fit_transform(X)
        
        for i_alg, (name, algorithm) in enumerate(algorithms.items()):
            t0 = time.time()
            try:
                if hasattr(algorithm, "fit_predict"):
                    y_pred = algorithm.fit_predict(X)
                else:
                    y_pred = algorithm.fit(X).predict(X)
            except Exception:
                y_pred = np.zeros(X.shape[0])
            t1 = time.time()
            
            # Plot
            ax = axes[i_dataset * len(algorithms) + i_alg]
            ax.scatter(X[:, 0], X[:, 1], c=y_pred, s=10, cmap='viridis')
            ax.set_title(f"{name}\n{(t1 - t0):.2f}s", fontsize=10)
            ax.set_xticks([])
            ax.set_yticks([])
    
    plt.tight_layout()
    plt.show()

# Main execution
datasets = generate_datasets()
algorithms = define_algorithms()
plot_results(datasets, algorithms)
